
# 📊 Visualizing Nutritional Food Deserts: Project Summary & Instructions

---

## 🧠 Project Vision
This project aims to visualize the concept of **nutritional food deserts**, focusing on how access to healthy food varies by location. Inspired by "Banking Desert" studies, the goal is to show how lack of access to nutritious options may correlate with **health outcomes** such as **diabetes** and **high blood pressure**.

### ✅ Core Objectives:
1. Analyze restaurants, convenience stores, and grocery stores to assess nutrition quality of food options.
2. Assign each location a **Nutritional Score** (from 0 = poor to 5 = excellent).
3. Map access to nutritious food geographically by ZIP code.
4. Visualize health outcome data (e.g., diabetes prevalence) by area.
5. Study the correlation between poor health outcomes and nutritional access.

---

## 🗺️ Planned Visualizations

### 1. Restaurant Nutritional Score Map
- **Type**: Geographic scatter or heat map
- **Color**: Red (0) to Green (5)
- **Location**: U.S. focus, starting with Georgia and Atlanta

### 2. Convenience Store Nutritional Score Map
- Similar layout as restaurants, but focuses on gas stations and corner stores

### 3. Grocery Store Nutritional Score Map
- Identifies if nutritious options are more available through grocery chains

### 4. Health Outcomes Heat Map
- **Type**: Choropleth / heat map
- **Metric**: Prevalence of diabetes, high blood pressure
- **Color**: Blue (low) to Red (high)

---

## 🧰 Toolchain for Implementation
These visualizations will be built first as **PNG mockups**, then implemented in:
- 📊 Google Sheets
- 🐍 Python (Plotly)
- 📈 Power BI
- 📉 Tableau

---

## 🛠️ Instructions for Implementation

### Step-by-Step:
1. Create **dummy data** for ZIP codes, coordinates, and nutrition scores
2. Use a **geographic base map** (USA → Georgia → Atlanta)
3. Overlay **points** or **choropleths** with color-coded nutrition scores
4. Use the **RdYlGn color scale** for accessibility
5. Add **ZIP code labels** for context
6. Export the map as **PNG** or **SVG**

---

## 🎨 Design Notes
- Use clear titles, legends, and labels
- Ensure all visual elements are readable and non-overlapping
- Maps should approximate state/city boundaries for geographic accuracy
- Consider colorblind-safe palettes
- Align design with the data literacy level of the audience


In [2]:
from __future__ import annotations
from typing import Optional, Literal
from datetime import datetime
from enum import Enum
from pydantic import BaseModel, Field, field_validator

In [4]:
# =========================
# Geographic / Admin Models
# =========================

class County(BaseModel):
    county_id: Optional[int] = None                 # PK (identity)
    title: str
    slug: str
    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)


class CityState(BaseModel):
    cs_id: Optional[int] = None                     # PK (identity)
    county_id: int                                  # FK -> County.county_id
    title: str
    slug: str
    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)

# =========================
# Category Models
# =========================

class AtlPlaceCat(BaseModel):
    atl_place_cat_id: Optional[int] = None          # PK (identity)
    title: str
    slug: str
    description: Optional[str] = None
    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)


class AtlPlaceSubCat(BaseModel):
    atl_place_sub_cat_id: Optional[int] = None      # PK (identity)
    atl_place_cat_id: int                           # FK -> AtlPlaceCat.atl_place_cat_id
    title: str
    slug: str
    description: Optional[str] = None
    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)

# =========================
# Place Model
# =========================

class OpenStatus(str, Enum):
    open = "open"
    closed = "closed"
    unknown = "unknown"


class AtlPlace(BaseModel):
    atl_place_id: Optional[int] = None              # PK (identity)
    title: str
    slug: str
    description: Optional[str] = None

    # Foreign keys
    atl_place_sub_cat_id: int                       # -> AtlPlaceSubCat
    # keywords_sub_cat_id: Optional[int] = None       # -> keywords_sub_category (not defined here)
    cs_id: Optional[int] = None                     # -> CityState.cs_id

    # Flags
    is_atl_verified: bool = False

    # Location
    address: Optional[str] = None
    lat: Optional[float] = None
    lng: Optional[float] = None
    open_status: OpenStatus = OpenStatus.unknown

    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)

    @field_validator("lat")
    @classmethod
    def _lat_range(cls, v):
        if v is None:
            return v
        if not (-90.0 <= v <= 90.0):
            raise ValueError("lat must be between -90 and 90")
        return v

    @field_validator("lng")
    @classmethod
    def _lng_range(cls, v):
        if v is None:
            return v
        if not (-180.0 <= v <= 180.0):
            raise ValueError("lng must be between -180 and 180")
        return v

# =========================
# Menu Models
# =========================

class PlaceType(str, Enum):
    food = "food"
    beverage = "beverage"
    restaurant = "restaurant"
   


class AtlPlaceMenu(BaseModel):
    atl_place_menu_id: Optional[int] = None         # PK (identity)
    atl_place_id: int                               # FK -> AtlPlace.atl_place_id
    place_type: PlaceType                           # restrict to food/beverage/restaurant
    title: str                                      # e.g., "Lunch Menu", "Drinks"
    description: Optional[str] = None
    is_active: bool = True

    # Optional denormalized metrics
    item_count: int = 0
    avg_nutrition_score: Optional[float] = None     # 0..5 (computed upstream)

    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)

    @field_validator("avg_nutrition_score")
    @classmethod
    def _score_range(cls, v):
        if v is None:
            return v
        if not (0.0 <= v <= 5.0):
            raise ValueError("avg_nutrition_score must be between 0 and 5")
        return v


class AtlPlaceMenuItem(BaseModel):
    atl_place_menu_item_id: Optional[int] = None    # PK (identity)
    atl_place_menu_id: int                          # FK -> AtlPlaceMenu.atl_place_menu_id

    # Core
    title: str                                      # e.g., "Grilled Chicken Bowl"
    description: Optional[str] = None

    # Pricing
    price: Optional[float] = None                   # numeric, no currency symbol
    currency: str = "USD"                           # ISO code
    is_available: bool = True

    # Nutrition (0..5) + macros (optional)
    nutrition_score: Optional[float] = None
    calories: Optional[int] = None
    protein_g: Optional[float] = None
    carbs_g: Optional[float] = None
    fat_g: Optional[float] = None
    tags: Optional[List[str]] = None                # e.g., ["vegan", "gluten-free"]

    created_at: datetime = Field(default_factory=datetime.utcnow)
    updated_at: datetime = Field(default_factory=datetime.utcnow)

    @field_validator("price")
    @classmethod
    def _price_nonneg(cls, v):
        if v is None:
            return v
        if v < 0:
            raise ValueError("price cannot be negative")
        return v

    @field_validator("nutrition_score")
    @classmethod
    def _item_score_range(cls, v):
        if v is None:
            return v
        if not (0.0 <= v <= 5.0):
            raise ValueError("nutrition_score must be between 0 and 5")
        return v
